In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import grblas
grblas.init('suitesparse')
from grblas import Matrix, Vector
from grblas import descriptor
from grblas import UnaryOp, BinaryOp, Monoid, Semiring
from grblas import io as gio

## Create and visualize a Matrix

In [ ]:
data = [
    [3,0,3,5,6,0,6,1,6,2,4,1],
    [0,1,2,2,2,3,3,4,4,5,5,6],
    [3,2,3,1,5,3,7,8,3,1,7,4]
]

In [ ]:
rows, cols, weights = data
m = Matrix.new_from_values(rows, cols, weights)
m
# Size of the sparse matrix is 7x7 with 12 non-zero elements of type INT64

In [ ]:
m.show()
# This is an adjacency matrix
# Reading along a row shows the out-nodes of a vertex
# Reading along a column shows the in-nodes of a vertex

In [ ]:
gio.draw(m)

## Create and visualize a Vector

In [ ]:
v = Vector.new_from_type(m.dtype, m.nrows)
v.element[1] = 0

In [ ]:
v.show()

## Single-source Shortest Path

This uses the **_min-plus_** semiring because we want to add the edges, then take the minimum length of available paths.

In [ ]:
# Create a vector and initialize a starting vertex (1) with a distance of zero
v = Vector.new_from_type(m.dtype, m.nrows)
v.element[1] = 0
v.show()

In [ ]:
m.show()

In [ ]:
# v @ m will give us one step in a Breadth-first search
w = Vector.new_from_existing(v)
w[:] = v.vxm(m, Semiring.MIN_PLUS)
w.show()

In [ ]:
# Look again at m and see that vertex 1 points to vertices 4 and 6 with the weights indicated
gio.draw(m)

We have the right semiring, but we already lost the initial distance=0 for vertex 1. How do we keep that information around as we step thru the BFS?

GraphBLAS has a builtin accumulator available for every operation.
Because it's C-based, you pass in the output object and it accumulates its existing values with the result, then returns itself.

In [ ]:
w = Vector.new_from_existing(v)
w[BinaryOp.MIN] = v.vxm(m, Semiring.MIN_PLUS)
w.show()
# Now we see that the zero distance for vertex 1 is preserved

Let's take another step

In [ ]:
w[BinaryOp.MIN] = w.vxm(m, Semiring.MIN_PLUS)
w.show()
# We see that the path to vertex 4 is now shorter. That's `min` doing its thing.
# Verify the other path distances from vertex 1 with at most two hops

In [ ]:
gio.draw(m)

The algorithm repeats until a new computation is the same as the previous result

In [ ]:
w = Vector.new_from_existing(v)
while True:
    w_old = Vector.new_from_existing(w)
    w[BinaryOp.MIN] = w.vxm(m, Semiring.MIN_PLUS)
    if w == w_old:
        break
w.show()

## Alternate solution without using accumulator

In the min_plus semiring, the "empty" value of a sparse matrix is not actually 0, but +infinity.

That way, `min(anything, +inf) = anything`, similar to the normal addition 0 of `add(anything, 0) = anything`.

A clever trick sets the diagonal of the matrix to all zeros. This makes it behave like the Identity matrix for the min_plus semiring.

Observe:

In [ ]:
m_ident = Matrix.new_from_values(range(7), range(7), [0]*7)
m_ident.show()

In [ ]:
v.rebuild_from_values([1], [0])
v.show()

In [ ]:
v[:] = v.vxm(m_ident, Semiring.MIN_PLUS)
v.show()
# See how it preserved v exactly

In [ ]:
# Let's try again
v.rebuild_from_values([0, 1, 4], [14, 0, 77])
v[:] = v.vxm(m_ident, Semiring.MIN_PLUS)
v.show()

So zeros along the diagonal preserve what you already have in `v` without adding any new path information. That's the behavior we want, so let's update `m` with zeros on the diagonal and repeat SSSP without using accumulators.

In [ ]:
for i in range(m.nrows):
    m.element[i, i] = 0
m.show()

In [ ]:
# Reset v
v.clear()
v.element[1] = 0
v.show()

In [ ]:
# Take one step (notice no accumulator is specified)
v[:] = v.vxm(m, Semiring.MIN_PLUS)
v.show()

In [ ]:
# Repeat until we're converged
while True:
    w = Vector.new_from_existing(v)
    v[:] = v.vxm(m, Semiring.MIN_PLUS)
    if v == w:
        break
v.show()

### And that's SSSP in 5 very readable lines of Python, thanks to GraphBLAS